<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Bot con NLTK utilizando un corpus de wikipedia


In [1]:
import json
import string
import random
import re
import urllib.request

import numpy as np

# Para leer y parsear el texto en HTML de wikipedia
import bs4 as bs

import nltk
# Descargar el diccionario
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zerba\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\zerba\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\zerba\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

### Datos
Se consumira los datos del artículo de wikipedia sobre el deporte "tennis" en ingles.

In [ ]:
# raw_html = urllib.request.urlopen('https://en.wikipedia.org/wiki/Harry_Potter')
# raw_html = raw_html.read()
# article_html = bs.BeautifulSoup(raw_html, 'lxml')
# article_paragraphs = article_html.find_all('p')

# article_text = ''

# for para in article_paragraphs:
#     article_text += para.text

# article_text = article_text.lower()

In [79]:
article_file = open(r"C:\Users\zerba\Downloads\Harry Potter.txt",encoding="utf8")

In [80]:
article_text = article_file.read()

In [ ]:
# Demos un vistazo
article_text

In [82]:
print("Cantidad de caracteres en la nota:", len(article_text))

Cantidad de caracteres en la nota: 446153


### 2 - Preprocesamiento
- Remover caracteres especiales
- Quitar espacios o saltos

In [83]:
text = re.sub(r'\[[0-9]*\]', ' ', article_text)
text = re.sub(r'\s+', ' ', text)

In [ ]:
# Demos un vistazo
text

In [14]:
print("Cantidad de caracteres en el texto:", len(text))

Cantidad de caracteres en el texto: 56448


### 3 - Dividir el texto en sentencias y en palabras

In [85]:
corpus = nltk.sent_tokenize(text)
words = nltk.word_tokenize(text)

In [ ]:
# Demos un vistazo
corpus[:10]

In [87]:
# Demos un vistazo
words[:20]

['They',
 'were',
 'the',
 'last',
 'people',
 'you',
 '’',
 'd',
 'expect',
 'to',
 'be',
 'involved',
 'in',
 'anything',
 'strange',
 'or',
 'mysterious',
 ',',
 'because',
 'they']

In [88]:
print("Vocabulario:", len(words))

Vocabulario: 102004


### 4 - Funciones de ayuda para limpiar y procesar el input del usuario
- Lematizar los tokens de la oración
- Quitar símbolos de puntuación

In [89]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def perform_lemmatization(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

punctuation_removal = dict((ord(punctuation), None) for punctuation in string.punctuation)

def get_processed_text(document):
    # 1 - reduce el texto a mínuscula
    # 2 - quitar los simbolos de puntuacion
    # 3 - realiza la tokenización
    # 4 - realiza la lematización
    return perform_lemmatization(nltk.word_tokenize(document.lower().translate(punctuation_removal)))

### 5 - Utilizar vectores TF-IDF y la similitud coseno construido con el corpus de wikipedia

In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def generate_response(user_input, corpus):
    response = ''
    # Sumar al corpus la pregunta del usuario para calcular
    # su cercania con otros documentos/sentencias
    corpus.append(user_input)

    # Crear un vectorizar TFIDF que quite las "stop words" del ingles y utilice
    # nuestra funcion para obtener los tokens lematizados "get_processed_text"
    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words='english')

    # Crear los vectores a partir del corpus
    all_word_vectors = word_vectorizer.fit_transform(corpus)

    # Calcular la similitud coseno entre todas los documentos excepto el agregado (el útlimo "-1")
    # NOTA: con los word embedings veremos más en detalle esta matriz de similitud
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)

    # Obtener el índice del vector más cercano a nuestra oración
    # --> descartando la similitud contra nuestor vector propio
    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0:
        response = "I am sorry, I could not understand you"
    else:
        response = corpus[similar_sentence_number]
    
    corpus.remove(user_input)
    return response

### 6 - Ensayar el sistema
El sistema intentará encontrar la parte del artículo que más se relaciona con nuestro texto de entrada. Sugerencias ensayar:
- Grand slam
- tournaments
- nadal
- artificial intelligence

In [91]:
# Se utilizará gradio para ensayar el bot
# Herramienta poderosa para crear interfaces rápidas para ensayar modelos
# https://gradio.app/
import sys
!{sys.executable} -m pip install gradio --quiet

"c:\Users\zerba\Documents\UBA\Repos\Repositorios" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [93]:
import gradio as gr

def bot_response(human_text):
    print(human_text)
    return generate_response(human_text.lower(), corpus)

iface = gr.Interface(
    fn=bot_response,
    inputs=["textbox"],
    outputs="text",
    layout="vertical")

iface.launch(debug=True)

c:\Users\zerba\Documents\UBA\Repos\Repositorios de Eloy\uba-env\lib\site-packages\gradio\deprecation.py:40: UserWarning: `layout` parameter is deprecated, and it has no effect
  warnings.warn(value)


Running on local URL:  http://127.0.0.1:7860/

To create a public link, set `share=True` in `launch()`.


plant crying


c:\Users\zerba\Documents\UBA\Repos\Repositorios de Eloy\uba-env\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


mandragora
Mandragora
Mandragora plant
voldemort
you know-who
you-know-who
Keyboard interruption in main thread... closing server.


(<gradio.routes.App at 0x150b32a1f10>, 'http://127.0.0.1:7860/', None)